In [3]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
# for dirname, _, filenames in os.walk('/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [ ]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

import warnings

import pickle
import time

import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans
from sklearn.multioutput import ClassifierChain
# from skmultilearn.problem_transform import ClassifierChain

import logging

from scipy.sparse import hstack

warnings.filterwarnings("ignore")
plt.style.use('bmh')
%matplotlib inline

In [ ]:
# AVG='micro'
y_true=np.array([1,0,1,1])
y_pred=np.array([0,1,1,0])
print(precision_score(y_pred,y_true))
print(recall_score(y_true,y_pred))
print(f1_score(y_true,y_pred))

In [ ]:
# Setting a random seed in order to keep the same random results each time I run the notebook
np.random.seed(seed=11)

In [4]:
# Importing the data into a dataframe
issues=pd.read_csv('../Curated issues-100.csv')
issues.head()

C:\Users\TEJAS\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Id,Title,Body,Created At,Updated At,Closed At,HTML URL,Status,is_locked,Role,Labels
0,603077341.0,b'[SPARK-31489][SPARK-31488][SQL] Translate da...,b'### What changes were proposed in this pull ...,2020-04-20T09:16:13Z,2020-04-20T09:24:37Z,NaN,https://github.com/apache/spark/pull/28272,open,FALSE,CONTRIBUTOR,b'SQL'
1,603063137.0,b'[SPARK-31495][SQL] Support formatted explain...,"b""<!--\r\nThanks for sending a pull request! ...",2020-04-20T08:56:30Z,2020-04-20T09:03:07Z,NaN,https://github.com/apache/spark/pull/28271,open,FALSE,CONTRIBUTOR,b'SQL'
2,602931061.0,b'[SPARK-31494][ML] flatten the result datafra...,b'### What changes were proposed in this pull ...,2020-04-20T04:32:21Z,2020-04-20T06:09:06Z,NaN,https://github.com/apache/spark/pull/28270,open,FALSE,CONTRIBUTOR,b'ML'
3,602921805.0,b'[SPARK-31493][SQL] Optimize InSet to In acco...,"b""<!--\r\nThanks for sending a pull request! ...",2020-04-20T04:02:56Z,2020-04-20T07:39:06Z,NaN,https://github.com/apache/spark/pull/28269,open,FALSE,CONTRIBUTOR,b'SQL'
4,602917700.0,b'[SPARK-31492][ML] flatten the result datafra...,b'### What changes were proposed in this pull ...,2020-04-20T03:48:38Z,2020-04-20T05:31:07Z,NaN,https://github.com/apache/spark/pull/28268,open,FALSE,CONTRIBUTOR,b'ML'


In [5]:
issues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164732 entries, 0 to 164731
Data columns (total 11 columns):
Id            164731 non-null float64
Title         164732 non-null object
Body          164732 non-null object
Created At    164730 non-null object
Updated At    164731 non-null object
Closed At     1 non-null object
HTML URL      164731 non-null object
Status        164731 non-null object
is_locked     164731 non-null object
Role          164730 non-null object
Labels        164730 non-null object
dtypes: float64(1), object(10)
memory usage: 13.8+ MB


In [ ]:
# checking=issues[issues['Body'].isna()]
# checking.info()

In [ ]:
issues.head()

In [ ]:
issues.drop(columns=['Created At', 'Updated At', 'Closed At',
#                      'HTML URL',
                     'Status','is_locked','Role'], inplace=True)

In [ ]:
issues.dropna(inplace=True)
issues.info()

In [ ]:
issues['Labels'].head(10)

In [ ]:
def utf82str(text):
    text=text.strip('b').strip("'")
    return text

def split_labels(labels):
    return labels.split(',')

In [ ]:
print(utf82str(issues['Title'].iloc[0]))
print((issues['Title'].iloc[0]))

In [ ]:
issues['Title']=issues['Title'].apply(lambda x: utf82str(x))
issues['Body']=issues['Body'].apply(lambda x: utf82str(x))
issues['Labels']=issues['Labels'].apply(lambda x: utf82str(x))

In [ ]:
plt.figure(figsize=(5, 5))
issues.isnull().mean(axis=0).plot.barh()
plt.title("Ratio of missing values per columns")
# new_df.isnull().mean(axis=0).head()

In [ ]:
print('Duplicate entries: {}'.format(issues.duplicated().sum()))
issues.drop_duplicates(inplace = True)

# **Processing the Labels**

1. convert to lowercase
2. remove left numbers
3. remove <>
4. replace : with / (if doesn't work try replacing / with :)
5. if \\xf0\\x9f\\x99\\x82 are there remove the label itself
6. remove [Type]


In [ ]:
issues['Labels']=issues['Labels'].apply(lambda x: x.lower())
issues.head()

In [ ]:
issues['Labels']=issues['Labels'].apply(lambda x: split_labels(x))
issues.head()

In [ ]:
def remove_leading_numbers(label_list):
    new_label_list=[]
    for label in label_list:
        new_label_list.append(label.lstrip('0123456789.-_:/ '))
        
    return new_label_list


def strip_numbers(label_list):
    new_label_list=[]
    for label in label_list:
        new_label_list.append(label.strip('0123456789.-_ '))
        
    return new_label_list

def remove_unwanted(label_list):
    new_label_list=[]
    for label in label_list:
#         temp=re.sub(r'\[?type\]?','',label)
        temp=re.sub(r"[<>]", "", label)
        new_label_list.append(temp)
        
    return new_label_list

def replace_char_with_char(label_list):
    new_label_list=[]
    for label in label_list:
        new_label_list.append(re.sub(r":", "/", label))
        
    return new_label_list

def remove_unencoded(label_list):
    new_label_list=[]
    for label in label_list:
        temp=re.sub(r"\\x",'',label)
        if temp==label:
            new_label_list.append(label)
        
    return new_label_list


In [ ]:
issues['Labels']=issues['Labels'].apply(lambda x: remove_unwanted(x))
issues.head()

In [ ]:
issues['Labels']=issues['Labels'].apply(lambda x: remove_leading_numbers(x))
issues.head()

In [ ]:
issues['Labels']=issues['Labels'].apply(lambda x: remove_unencoded(x))
issues.head()

In [ ]:
issues['Labels']=issues['Labels'].apply(lambda x: replace_char_with_char(x))
issues.head()

In [ ]:
issues['Labels'].head(100)

In [ ]:
all_labels = [item for sublist in issues['Labels'].values for item in sublist]
print(len(all_labels))
# print(all_labels[13546:14600])

In [ ]:
# removing null labels from the label list

temp=pd.DataFrame(all_labels,columns=['labels'])
temp=temp[temp['labels']!='']
all_labels=list(temp['labels'])
print(len(all_labels))

In [ ]:
all_labels[:100]

In [ ]:
unique_labels=list(set(all_labels))
print(len(unique_labels))

In [ ]:
# flat_list = [item for sublist in new_df['Tags'].values for item in sublist]

keywords = nltk.FreqDist(all_labels)

# print(keywords.items())
keywords = nltk.FreqDist(keywords)
# print(keywords.most_common(5))
frequencies_words = keywords.most_common(100)
labels_features = [word[0] for word in frequencies_words]

In [ ]:
check=keywords.most_common(100)
for word in check:
    print(word[0])

In [ ]:
print(len(labels_features))
feature_check=' '.join(labels_features)
# print(feature_check)
# print('ing ci_' in feature_check)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
keywords.plot(10, cumulative=False)

In [ ]:
def most_common(labels):
    labels_filtered = []
    for i in range(0, len(labels)):
        if labels[i] in labels_features:
            labels_filtered.append(labels[i])
    return labels_filtered

In [ ]:
issues['Labels']=issues['Labels'].apply(lambda x: most_common(x))
issues['Labels']=issues['Labels'].apply(lambda x: x if len(x)>0 else None)

In [ ]:
print(issues.shape)
issues.head()

In [ ]:
issues['Labels'].isna().sum()

In [ ]:
issues.dropna(subset=['Labels'], inplace=True)

In [ ]:
issues.shape

In [ ]:
issues.head()

In [ ]:
issues['Body'].iloc[123]

# **Testing Body preprocessing**

In [ ]:
# getting rid of code snippets,error logs and shit
def remove_screenshots(text):
    return re.sub(r"```(.*?)```", '__________', temp, flags=re.MULTILINE)

# getting rid of people tags
def remove_people_tags(text):
    return re.sub(r"@[a-zA-Z0-9_.-]+",' ',text)

# getting rid of urls
def remove_url(text):
    return re.sub(r"https?\S+", ' ', text, flags=re.MULTILINE)

# getting rid of \n and \r
def remove_escape_seqs(text):
    return re.sub(r"\\r|\\n", ' ', text, flags=re.MULTILINE)

# removing numbers
def remove_numbers(text):
    return re.sub(r"[0-9]+", ' ', text, flags=re.MULTILINE)

# remove single letters
def remove_single_letters(sentence):
    return ' '.join( [w for w in sentence.split() if len(w)>1] )





In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [ ]:
token=ToktokTokenizer()
punct = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
punctuation

In [ ]:
def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']

def clean_punct(text): 
    words=token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    remove_punctuation = str.maketrans(' ', ' ', punct)
    for w in words:
        if w in labels_features:
            punctuation_filtered.append(w)
        else:
            punctuation_filtered.append(regex.sub(' ', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))


In [ ]:
issues['Body'].iloc[12345]

In [ ]:
sample=0
temp=issues['Body'].iloc[sample]
print(temp)
print(issues['HTML URL'].iloc[sample])

# **Actual Body and title processing**

In [ ]:
issues['Title'] = issues['Title'].apply(lambda x: BeautifulSoup(x).get_text()) 
issues['Title'] = issues['Title'].apply(lambda x: remove_people_tags(x)) 
issues['Title'] = issues['Title'].apply(lambda x: remove_url(x))
issues['Title'] = issues['Title'].apply(lambda x: remove_escape_seqs(x)) 
issues['Title'] = issues['Title'].apply(lambda x: clean_text(x)) 
issues['Title'] = issues['Title'].apply(lambda x: clean_punct(x)) 
issues['Title'] = issues['Title'].apply(lambda x: remove_numbers(x)) 
issues['Title'] = issues['Title'].apply(lambda x: remove_single_letters(x)) 

In [ ]:
issues['Body'] = issues['Body'].apply(lambda x: BeautifulSoup(x).get_text()) 
# issues['Body'] = issues['Body'].apply(lambda x: remove_screenshots(x))
issues['Body'] = issues['Body'].apply(lambda x: remove_people_tags(x))
issues['Body'] = issues['Body'].apply(lambda x: remove_url(x))
issues['Body'] = issues['Body'].apply(lambda x: remove_escape_seqs(x))
issues['Body'] = issues['Body'].apply(lambda x: clean_text(x))
issues['Body'] = issues['Body'].apply(lambda x: clean_punct(x))
issues['Body'] = issues['Body'].apply(lambda x: remove_numbers(x))
issues['Body'] = issues['Body'].apply(lambda x: remove_single_letters(x)) 



In [ ]:
issues['Body'].iloc[0]

**add more stop words<br>
dont remove labels from title or body**

In [ ]:
lemma=WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
# stop_words= list(stop_words)+['what','pr','is','this','does','it','we',
#                               'eg','cc','module','happens','happened',
#                               'issue','why','environment','os','etc']

def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))


def stopWordsRemove(text):
#     stop_words = set(stopwords.words("english"))
    words=token.tokenize(text)
    filtered = [w for w in words if w in feature_check or not w in stop_words]
    
    return ' '.join(map(str, filtered))

In [ ]:
issues['Title'] = issues['Title'].apply(lambda x: lemitizeWords(x)) 
issues['Title'] = issues['Title'].apply(lambda x: stopWordsRemove(x)) 
issues['Body'] = issues['Body'].apply(lambda x: lemitizeWords(x)) 
issues['Body'] = issues['Body'].apply(lambda x: stopWordsRemove(x)) 

In [ ]:
issues.info()

In [ ]:
issues['Body'].iloc[0]

In [ ]:
no_topics = 20
text = issues['Body']

In [ ]:
vectorizer_train = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       token_pattern=r"(?u)\S\S+", # Need to repeat token pattern
                                       max_features=1000)

In [ ]:
TF_IDF_matrix = vectorizer_train.fit_transform(text)

In [ ]:
len(vectorizer_train.vocabulary_)

In [ ]:
# print(TF_IDF_matrix.shape)
temp=TF_IDF_matrix
dense=temp.todense()
# print(dense)
denselist=dense.tolist()
# print(denselist)

tfidf=pd.DataFrame(denselist,columns=vectorizer_train.get_feature_names())
print(len(tfidf))
tfidf.head()

In [ ]:
tfidf.shape

In [ ]:
print(vectorizer_train.get_feature_names())

In [ ]:
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50,random_state=11).fit(TF_IDF_matrix)

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("--------------------------------------------")
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        print("--------------------------------------------")
        

no_top_words = 10
display_topics(lda, vectorizer_train.get_feature_names(), no_top_words)

In [ ]:
issues.info()

In [ ]:
X1 = issues['Body']
X2 = issues['Title']
y = issues['Labels']

In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
y_bin = multilabel_binarizer.fit_transform(y)

In [ ]:
y_bin.shape

In [ ]:
vectorizer_X1 = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       token_pattern=r"(?u)\S\S+",
                                       max_features=1000)

vectorizer_X2 = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       token_pattern=r"(?u)\S\S+",
                                       max_features=1000)

In [ ]:
X1_tfidf = vectorizer_X1.fit_transform(X1)
X2_tfidf = vectorizer_X2.fit_transform(X2)

In [ ]:
print(len(vectorizer_X1.vocabulary_))
print(len(vectorizer_X2.vocabulary_))

In [ ]:
print(X1_tfidf.shape)
print(X2_tfidf.shape)
X_tfidf = hstack([X1_tfidf,X2_tfidf])
# X_tfidf=X2_tfidf
print(X_tfidf.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_bin, test_size = 0.2, random_state = 0) # Do 80/20 split

In [ ]:
def avg_jacard(y_true,y_pred):
    '''
    see https://en.wikipedia.org/wiki/Multi-label_classification#Statistics_and_evaluation_metrics
    '''
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    
    return jacard.mean()*100

def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("Precision: {}".format(precision_score(y_test, y_pred,average='samples')*100))
    print("Recall: {}".format(recall_score(y_test, y_pred,average='samples')*100))
    print("F score: {}".format(f1_score(y_test, y_pred,average='samples')*100))
    print("Hamming loss: {}".format(hamming_loss(y_test, y_pred)*100))
    
    print("---")    

In [ ]:
# # 46 54
dummy = DummyClassifier()
sgd = SGDClassifier()
lr = LogisticRegression()
mn = MultinomialNB()
svc = LinearSVC()  # 10chains 0.15thresh f1 60.3
                    # 15chains 0.15thresh f1 60.6
perceptron = Perceptron()  # 10chains 0.25thresh f1 56
pac = PassiveAggressiveClassifier()  # 10chains 0.25thresh f1 58

for classifier in [dummy, sgd, lr, mn, svc, perceptron, pac]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)

In [ ]:
# Fit an independent logistic regression model for each class using the
# OneVsRestClassifier wrapper.
base_lr = LinearSVC()
ovr = OneVsRestClassifier(base_lr)
ovr.fit(X_train, y_train)
Y_pred_ovr = ovr.predict(X_test)
ovr_f1_score = f1_score(y_test, Y_pred_ovr, average='samples')
ovr_f1_score

In [ ]:
# Fit an ensemble of logistic regression classifier chains and take the
# take the average prediction of all the chains.
chains = [ClassifierChain(base_lr, order='random', random_state=i) for i in range(10)]
i=1
for chain in chains:
    print(i)
    i+=1
    chain.fit(X_train, y_train)


In [ ]:
Y_pred_chains = np.array([chain.predict(X_test) for chain in chains])
Y_pred_ensemble = Y_pred_chains.mean(axis=0)
thresholds=np.arange(0.01,0.9,.01)

for thresh in thresholds:
    ensemble_f1_score = f1_score(y_test,
                                       Y_pred_ensemble >= thresh,
                                       average='samples')
    print(np.round(thresh,2),ensemble_f1_score)

In [ ]:
threshold=0.15
# y_pred =model.predict(X_test)
# print(y_pred[0])
# print(y_test[0])
y_pred_new=np.zeros(Y_pred_ensemble.shape)

for i in range(Y_pred_ensemble.shape[0]):
    for j in range(Y_pred_ensemble.shape[1]):
        
        if Y_pred_ensemble[i,j]>=threshold:
            y_pred_new[i,j]=1
                   
ensemble_f1_score = f1_score(y_test,
                                       Y_pred_ensemble >= threshold,
                                       average='samples')

In [ ]:
chain_f1_scores = [f1_score(y_test, Y_pred_chain >= threshold,
                                      average='samples')
                        for Y_pred_chain in Y_pred_chains]


model_scores = [ovr_f1_score] + chain_f1_scores
model_scores.append(ensemble_f1_score)


In [ ]:

model_names = ('Independent',
               'Chain 1',
               'Chain 2',
               'Chain 3',
               'Chain 4',
               'Chain 5',
               'Chain 6',
               'Chain 7',
               'Chain 8',
               'Chain 9',
               'Chain 10',
               'Ensemble')

x_pos = np.arange(len(model_names))

# Plot the Jaccard similarity scores for the independent model, each of the
# chains, and the ensemble (note that the vertical axis on this plot does
# not begin at 0).

fig, ax = plt.subplots(figsize=(7, 4))
ax.grid(True)
ax.set_title('Classifier Chain Ensemble Performance Comparison')
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, rotation='vertical')
ax.set_ylabel('F1 Similarity Score')
ax.set_ylim([min(model_scores) * .9, max(model_scores) * 1.1])
colors = ['r'] + ['b'] * len(chain_f1_scores) + ['g']
ax.bar(x_pos, model_scores, alpha=0.5, color=colors)
plt.tight_layout()
plt.show()

In [ ]:
print_score(y_pred_new,ovr)

In [ ]:
print(y_pred_new.shape)
print(y_test.shape)

In [ ]:
test_sample=123

groundtruth=[]
for i in range(len(y_test[test_sample])):
    if y_test[test_sample][i]==1:
        groundtruth.append(multilabel_binarizer.classes_[i])
        
predicted=y_pred_new[test_sample]
# print(len(predicted))
predicted_labels=[]
for i in range(len(predicted)):
    if predicted[i]==1:
        predicted_labels.append(multilabel_binarizer.classes_[i])

print(groundtruth)
print(predicted_labels)

print('Common labels : ',list(set(groundtruth) & set(predicted_labels)))

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Neural network
# 200 100 2000 feats thresh 0.5| j47 p55 f53
# 200 100 2000 feats thresh 0.05| j49 p54 f56
# 400 100 2000 feats thresh 0.05| j50 p54 f57
# 200 100 2000 feats thresh 0.1| j50 p55 f56
# 100     2000 feats thresh 0.25| j49 p54 f56
# 400 200 100 2000 feats thresh 0.1 |j49 p55 f55
model = Sequential()
model.add(Dense(4000, input_dim=2000, activation='relu'))
# model.add(Dense(200, activation='relu'))
model.add(Dense(100,
#                 input_dim=2000,
                activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=256)

In [ ]:
score=model.evaluate(X_test,y_test)
print(score)


In [ ]:
y_pred =model.predict(X_test)
thresholds=np.arange(0.01,0.9,.01)

for thresh in thresholds:
    fscore = f1_score(y_test,y_pred >= thresh,
                                       average='samples')
    print(np.round(thresh,2),fscore)

In [ ]:
threshold=0.08
# print(y_pred[0])
# print(y_test[0])
y_pred_new=np.zeros(y_pred.shape)

for i in range(y_pred.shape[0]):
    for j in range(y_pred.shape[1]):
        
        if y_pred[i,j]>=threshold:
            y_pred_new[i,j]=1
                   

In [ ]:
print_score(y_pred_new,model)

In [ ]:
issues.head(1)

In [ ]:
# # 400,100 : j49 p57 f54
# # 200,100 : j50 p60
# # 100     : j48 p56 f53
# # 100,100 : j48 p56 f53
# mlpc = MLPClassifier(hidden_layer_sizes=(200,100),
#                      activation='relu',
#                      verbose=True,
#                      early_stopping=True,
#                      validation_fraction=0.1,
#                      n_iter_no_change=10,
#                      learning_rate='adaptive',
#                      max_iter=25)
# mlpc.fit(X_train, y_train)

# y_pred = mlpc.predict(X_test)

# print_score(y_pred, mlpc)

In [ ]:
y_test.shape

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

print_score(y_pred, rfc)

In [ ]:
print(multilabel_binarizer.classes_)

In [ ]:
test_sample=189

groundtruth=[]
for i in range(len(y_test[test_sample])):
    if y_test[test_sample][i]==1:
        groundtruth.append(multilabel_binarizer.classes_[i])
        
predicted=ovr.predict(X_test[test_sample])[0]
# print(len(predicted))
predicted_labels=[]
for i in range(len(predicted)):
    if predicted[i]==1:
        predicted_labels.append(multilabel_binarizer.classes_[i])

print(groundtruth)
print(predicted_labels)

print('Common labels : ',list(set(groundtruth) & set(predicted_labels)))

In [ ]:
for i in range(y_train.shape[1]):
    print(multilabel_binarizer.classes_[i])
    print(confusion_matrix(y_test[:,i], y_pred[:,i]))
    print("")

In [ ]:
def print_top10(feature_names, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[i])[-10:]
        print("--------------------------------------------")
        print("%s: %s" % (class_label,
              " ".join(feature_names[j] for j in top10)))
        print("--------------------------------------------")

In [ ]:
feature_names = vectorizer_X1.get_feature_names() + vectorizer_X2.get_feature_names()
# feature_names = vectorizer_X2.get_feature_names()


In [ ]:
print_top10(feature_names, ovr, multilabel_binarizer.classes_)